<a href="https://colab.research.google.com/github/rohbot25/Erohan_Repository/blob/main/CollisionTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install taichi==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00


In [ ]:
import taichi as ti
import numpy as np
import os
import math

max_steps = 300
dt = 0.01
learning_rate = 0.1
gravity = 0.0
damping = 0.0
stiffness = 1000.0
ground_height = 0.1
sim_size = 512

soil_particles = 2

real = ti.f32

soil = []
mass = []
#for i in range(soil_particles):
  #soil.append([np.random.random(),np.random.random()* 0.1 + 0.1])
  #mass.append(int(np.random.random() * 8) + 3)
soil.append([0.0,1.0])
soil.append([1.0,0.5])
mass.append(50)
mass.append(35)
ti.init( default_fp = real )

vec = lambda: ti.Vector.field(2, dtype = real)

positions = vec()
velocities = vec()
masses = ti.field(ti.i32)

# Declare fields for variables to debug
normals = vec()
tangents = vec()
v1ns = ti.field(dtype=real)
v1ts = ti.field(dtype=real)
v2ns = ti.field(dtype=real)
v2ts = ti.field(dtype=real)
v1n2s = ti.field(dtype=real)
v2n2s = ti.field(dtype=real)
vec1ns = vec()
vec1ts = vec()
vec2ns = vec()
vec2ts = vec()
overlaps = ti.field(dtype=real)

ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(normals)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(tangents)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1n2s)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2n2s)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(overlaps)



ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(positions)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(velocities)
ti.root.dense(ti.i, soil_particles).place(masses)


# ------------------------------------------

def Draw(frameOffset):
  for timeStep in range(0,max_steps):

    gui = ti.GUI("Robot" , (512,512), background_color = 0xFFFFFF, show_gui=False)

    #draw the floor
    gui.line( begin = ( 0, ground_height) ,
              end = (1,ground_height) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (0,0) ,
              end = (0,1) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (1,0) ,
              end = (1,1) ,
              color = 0x0,
              radius = 1)
    #draw the soil
    for particle in range(soil_particles):
      x = positions[timeStep,particle][0]
      y = positions[timeStep,particle][1]
      if(particle == 0):
        gui.circle ( (x,y), color = 0x0000FF , radius = masses[particle])
      else:
        gui.circle ( (x,y), color = 0xFF0000 , radius = masses[particle] )

    gui.show( 'test' + str(frameOffset + timeStep) + '.png')

# ----------------------------------------
def Initialize():

  for objectIndex in range(soil_particles):

    positions[0,objectIndex] = soil[objectIndex]

    velocities[0,0] = [-1,-0.25]
    velocities[0,1] = [1,0]

    masses[objectIndex] = mass[objectIndex]

# ----------------------------------------
def Simulate():

  for timeStep in range(1,max_steps):

    Step_One(timeStep)
#----------------------------------------

@ti.kernel
def Simulate_Objects(timeStep: ti.i32):

  for objA in range(soil_particles):

    oldPosition = positions[timeStep-1, objA]

    newVelocity = velocities[timeStep-1, objA] + \
    dt * gravity * ti.Vector([0,1])

    newPosition = oldPosition + dt * newVelocity
    collision = False
    for objB in range(soil_particles):
      if (objB != objA):
        posA = positions[timeStep-1,objA]
        posB = positions[timeStep-1,objB]
        dist = posA - posB
        length = dist.norm()

        radiusA = masses[objA] / sim_size
        radiusB = masses[objB] / sim_size
        radiiSum = radiusA + radiusB
        if (length < radiiSum):
          collision = True
          normal = dist.normalized()
          tangent = ti.Vector([-normal[1], normal[0]])

          v1n = ti.Vector.dot(newVelocity, normal)
          v1t = ti.Vector.dot(newVelocity, tangent)
          v2n = ti.Vector.dot(velocities[timeStep-1,objB], normal)
          v2t = ti.Vector.dot(velocities[timeStep-1,objB], tangent)

          v1n2 = (v1n * (masses[objA] - masses[objB]) + 2 * masses[objB] * v2n) / (masses[objA] + masses[objB])
          v2n2 = (v2n * (masses[objB] - masses[objA]) + 2 * masses[objA] * v1n) / (masses[objA] + masses[objB])

          vec1n = v1n2 * normal
          vec1t = v1t * tangent
          vec2n = v2n2 * normal
          vec2t = v2t * tangent

          velocities[timeStep,objA] = (vec1n + vec1t) * (1-damping)
          velocities[timeStep,objB] = (vec2n + vec2t) * (1-damping)
          overlap = (radiiSum - length) / 2
          newPosition = positions[timeStep-1,objA] + overlap * normal
          positions[timeStep, objB] = positions[timeStep-1,objB] - overlap * normal

          while length < radiiSum:
            newPosition += overlap * normal
            positions[timeStep, objB] -= overlap * normal
            length += overlap * 2

    particle_radius = masses[objA] / sim_size
    if newPosition[0] > 1 - particle_radius:
          newPosition[0] = 1 - particle_radius
          newVelocity[0] *= -1 * (1-damping)

    if newPosition[0] < particle_radius:
          newPosition[0] = particle_radius
          newVelocity[0] *= -1 * (1-damping)

    if newPosition[1] < ground_height + particle_radius:
          newPosition[1] = ground_height + particle_radius
          newVelocity[1] *= -1 * (1-damping)

    if newPosition[1] > 1 - particle_radius:
          newPosition[1] = 1 - particle_radius
          newVelocity[1] *= -1 * (1-damping)

    if not collision:
      velocities[timeStep,objA] = newVelocity
      positions[timeStep,objA] = newPosition

def Step_One(timeStep: ti.i32):

  Simulate_Objects(timeStep)


# ----------------------------------------

def Make_Movie():

  os.system("rm movie.mp4")
  os.system(" ffmpeg -i test%d.png movie.mp4")

# ----------------------------------------


# ---------- Main body of code

Initialize()

Simulate()

os.system("rm *.png")

Draw(0)


Make_Movie()

# Debug printing function
def print_debug_info():
    for timeStep in range(1, max_steps):
        for objA in range(soil_particles):
            print(f"Time Step: {timeStep}, Object: {objA}")
            print(f"Normal: {normals[timeStep, objA]}")
            print(f"Tangent: {tangents[timeStep, objA]}")
            print(f"v1n: {v1ns[timeStep, objA]}")
            print(f"v1t: {v1ts[timeStep, objA]}")
            print(f"v2n: {v2ns[timeStep, objA]}")
            print(f"v2t: {v2ts[timeStep, objA]}")
            print(f"v1n2: {v1n2s[timeStep, objA]}")
            print(f"v2n2: {v2n2s[timeStep, objA]}")
            print(f"vec1n: {vec1ns[timeStep, objA]}")
            print(f"vec1t: {vec1ts[timeStep, objA]}")
            print(f"vec2n: {vec2ns[timeStep, objA]}")
            print(f"vec2t: {vec2ts[timeStep, objA]}")
            print(f"Overlap: {overlaps[timeStep, objA]}")
            print(f"Updated Position: {positions[timeStep, objA]}")
            print(f"Updated Velocity: {velocities[timeStep, objA]}")
            print("------------------")

# Call this function after Make_Movie() in your main code
#print_debug_info()

#watch movie
from IPython.display import HTML
from base64 import b64encode
mp4 = open('movie.mp4', 'rb').read()
data_url = "data:video/mp4;base64,"+ b64encode(mp4).decode()

HTML('<video width=sim_size controls> <source src="%s" type="video/mp4"></video>' % data_url)



[Taichi] Starting on arch=x64
